# Trabalhando com coordenadas celestes no WCS 1: Especificando, lendo e plotando

## Autores

Kris Stern, Kelle Cruz, Lia Corrales, David Shupe, Adrian Price-Whelan

## Tradução

Josué Cavalcante

## Objetivos de aprendizagem

1. Demonstrar duas maneiras de construir o objeto `astropy.wcs.WCS`
2. Mostre uma imagem da nebulosa Helix com RA e DEC rotulados
3. Plotar uma escala de barras em uma imagem com informações WCS

## Palavras chaves

WCS, coordenadas, matplotlib

## Conteúdo complementar
1. "An Introduction to Modern Astrophysics" ([Carroll & Ostlie](https://ui.adsabs.harvard.edu/abs/2006ima..book.....C/abstract))
2. [FITS WCS page at GSFC](https://fits.gsfc.nasa.gov/fits_wcs.html)

## Sumário

Esta série de tutoriais tem como objetivo mostrar como o conteúdo do Capítulo 1 de "An Introduction to Modern Astrophysics" de Carroll e Ostlie pode ser aplicado a situações de pesquisa em astrofísica da vida real, usando ferramentas do ecossistema Astropy. Apresentaremos duas abordagens diferentes para construir um objeto `astropy.wcs.WCS`, que contém metadados que (neste caso) definem um mapeamento entre as coordenadas da imagem e as coordenadas do céu. O subpacote `astropy.wcs` está em conformidade com os padrões do FITS World Coordinate System (WCS) usado extensivamente pela comunidade de pesquisa em astronomia. Criaremos um WCS 2D para uma imagem da icônica nebulosa Helix (uma nebulosa planetária) e exibiremos uma imagem da nebulosa com as coordenadas do céu (aqui, equatorial, ICRS RA e Dec) rotuladas. Finalmente, sobreporemos uma barra de escala na imagem da nebulosa Helix usando WCS para dar ao leitor uma noção do tamanho angular da imagem.

In [ ]:
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt

## Seção 1: Duas maneiras de criar um objeto `astropy.wcs.WCS`

*Coordenadas mundiais* servem para localizar uma medição em algum espaço de parâmetros multidimensional. Um Sistema de Coordenadas Mundiais (WCS) especifica as coordenadas físicas, ou mundiais, a serem anexadas a cada pixel ou voxel de uma imagem ou matriz N-dimensional. Um [conjunto elaborado de padrões e convenções](https://fits.gsfc.nasa.gov/fits_wcs.html) foi desenvolvido para o formato Flexible Image Transport System (FITS) ([Wells et al. 1981](https: //ui.adsabs.harvard.edu/abs/1981A&AS...44..363W/abstract)). Um exemplo típico do WCS é especificar a Ascensão Reta (RA) e a Declinação (Dec) no céu associada a uma determinada localização de pixel ou spaxel em uma imagem celestial bidimensional ([Greisen & Calabretta 2002](https://ui .adsabs.harvard.edu/abs/2002A&A...395.1061G/abstract); [Calabretta e Greisen 2002](https://ui.adsabs.harvard.edu/abs/2002A&A...395.1077C/abstract)) .

O [subpacote `astropy.wcs`](https://docs.astropy.org/en/stable/wcs/) implementa padrões e convenções FITS para Sistemas de Coordenadas Mundiais. Usando o objeto `astropy.wcs.WCS` e `matplotlib`, podemos gerar imagens do céu que possuem eixos rotulados com coordenadas como ascensão reta (RA) e declinação (Dec). Isso requer selecionar as projeções apropriadas para `matplotlib` e fornecer um objeto `astropy.visualization.WCSAxes`.

Existem duas maneiras principais de inicializar um objeto `WCS`: com um dicionário Python (ou objeto semelhante a um dicionário, como um cabeçalho de arquivo FITS) ou com listas Python. Neste conjunto de exemplos, inicializaremos um objeto `astropy.wcs.WCS` com duas dimensões, como seria necessário para representar uma imagem.

O padrão WCS define um conjunto de palavras-chave que são usadas para representar o sistema de coordenadas mundiais para um determinado conjunto de dados (por exemplo, imagem). Aqui está uma lista das palavras-chave essenciais do WCS e seus usos; Em cada caso, o inteiro $n$ denota o eixo dimensional (começando com 1) ao qual a palavra-chave está sendo aplicada. Em nossos exemplos abaixo, teremos duas dimensões de imagem (eixos), então $n$ será 1 ou 2.
* **CRVALn**: o valor da coordenada em um ponto de referência (por exemplo, valor RA e DEC em graus)
* **CRPIXn**: a localização do pixel do ponto de referência (por exemplo, CRPIX1=1, CRPIX2=1 descreve o centro de um pixel de canto)
* **CDELTn**: o incremento de coordenadas no ponto de referência (por exemplo, a diferença no valor 'RA' do pixel de referência para seu vizinho ao longo do eixo RA)
* **CTYPEn**: uma sequência de 8 caracteres que descreve o tipo de eixo (por exemplo, 'RA---TAN' e 'DEC---TAN' descrevem a projeção do céu do plano tangente típica que os astrônomos usam)
* **CUNITn**: uma string descrevendo a unidade para cada eixo (se não for especificada, a unidade padrão é graus.)
* **NAXISn**: um inteiro que define o número de pixels em cada eixo

Algumas boas referências do padrão WCS podem ser encontradas [aqui](https://fits.gsfc.nasa.gov/fits_wcs.html).

### Método 1: Construindo um objeto WCS com um dicionário

Uma maneira de definir um objeto Astropy `WCS` é construir um dicionário contendo todas as informações essenciais (ou seja, especificando valores para as palavras-chave WCS listadas acima) que mapeiam o espaço de coordenadas de pixel para o espaço de coordenadas mundiais.

Neste exemplo, definimos dois eixos de coordenadas com:
* Uma projeção Gnomônica (plano tangente), que corresponde ao sistema de coordenadas RA/Dec
* Um local de referência de (RA,DEC) = (337,52, -20,83), conforme definido pelas chaves **CRVALn**
* O pixel no valor da coordenada (1,1) como local de referência (chaves **CRPIXn**)
* Unidades de graus (**CUNITn = 'deg'**)
* Tamanhos de pixel de 1 x 1 segundo de arco (**CDELTn = 0,002778** em graus)
* Um tamanho de imagem de 1024 x 1024 pixels (chave **NAXISn**)

In [ ]:
wcs_input_dict = {
    'CTYPE1': 'RA---TAN', 
    'CUNIT1': 'deg', 
    'CDELT1': -0.0002777777778, 
    'CRPIX1': 1, 
    'CRVAL1': 337.5202808, 
    'NAXIS1': 1024,
    'CTYPE2': 'DEC--TAN', 
    'CUNIT2': 'deg', 
    'CDELT2': 0.0002777777778, 
    'CRPIX2': 1, 
    'CRVAL2': -20.833333059999998, 
    'NAXIS2': 1024
}
wcs_helix_dict = WCS(wcs_input_dict)

Agora vamos imprimir o objeto `WCS` definido com um dicionário Python para verificar seu conteúdo:

In [ ]:
wcs_helix_dict # To check output

Nesta demonstração (abaixo), assumimos que conhecemos todos os valores de palavras-chave relevantes do WCS a serem especificados. Normalmente, no entanto, contaremos com software para produzir esses valores para nós. Por exemplo, as informações WCS são mais frequentemente incluídas automaticamente em arquivos FITS produzidos por software usado para tirar imagens com a maioria dos instrumentos em observatórios astronômicos. Nos casos em que as informações do WCS são fornecidas para nós em um arquivo FITS, elas normalmente são incluídas em um cabeçalho FITS, que, quando lido em Python, atua como um objeto de dicionário. Demonstramos isso mais adiante neste tutorial.

### Método 2: Crie um objeto WCS vazio antes de atribuir valores

Alternativamente, poderíamos inicializar o objeto `astropy.wcs.WCS` e atribuir os valores da palavra-chave com listas correspondentes a cada eixo respectivo.

In [ ]:
wcs_helix_list = WCS(naxis=2)
wcs_helix_list.wcs.crpix = [1, 1]
wcs_helix_list.wcs.crval = [337.5202808, -20.833333059999998]
wcs_helix_list.wcs.cunit = ["deg", "deg"]
wcs_helix_list.wcs.ctype = ["RA---TAN", "DEC--TAN"]
wcs_helix_list.wcs.cdelt = [-0.0002777777778, 0.0002777777778]

Vamos imprimir o objeto `WCS` mais uma vez para verificar como nossos valores foram atribuídos.

In [ ]:
wcs_helix_list # To check output

Observe que quando inicializamos o objeto WCS desta forma, os valores `NAXIS` são definidos como 0. Para atribuir coordenadas à nossa imagem, precisaremos corrigir a forma do array de objetos `WCS` para que corresponda à nossa imagem. Podemos fazer isso atribuindo um valor ao atributo `array_shape` do objeto `WCS`:

In [ ]:
wcs_helix_list.array_shape = [1024, 1024]

Agora, quando imprimimos o objeto `WCS`, podemos ver que os valores `NAXIS` foram atualizados do tamanho padrão de 0 a 1024.

In [ ]:
wcs_helix_list

## Seção 2: Mostre uma imagem da nebulosa Helix com RA e Dec rotulados

Na maioria das vezes, podemos obter o objeto `astropy.wcs.WCS` necessário do cabeçalho do arquivo FITS de um telescópio ou banco de dados astronômico. Este processo é descrito abaixo.

### Passo 1: Leia o arquivo FITS

Vamos ler o arquivo FITS contendo uma imagem da nebulosa Helix do repositório `astropy-data` GitHub usando o subpacote `astropy.io.fits`. A função `astropy.io.fits.open()` carregará o conteúdo de um arquivo FITS no Python e aceita um caminho de arquivo local ou uma URL (como demonstrado aqui). Esta imagem (arquivo FITS) foi originalmente acessada do [Digitized Sky Survey](https://archive.eso.org/dss/dss), mas é fornecida no repositório `astropy-data` por conveniência:

In [ ]:
header_data_unit_list = fits.open('https://github.com/astropy/astropy-data/raw/6d92878d18e970ce6497b70a9253f65c925978bf/tutorials/celestial-coords1/tailored_dss.22.29.38.50-20.50.13_60arcmin.fits')

Os arquivos FITS são um formato de arquivo binário usado principalmente por astrônomos e podem conter informações organizadas em muitas "extensões", que contêm informações de cabeçalho (por exemplo, metadados) e dados (por exemplo, dados de imagem). Podemos verificar quantas extensões existem em um arquivo FITS, bem como visualizar um resumo do conteúdo de cada extensão, imprimindo as informações do objeto FITS.

In [ ]:
header_data_unit_list.info()

Isso nos mostra que nosso arquivo FITS contém apenas uma extensão, rotulada 'PRIMARY' (ou extensão número 0). Copiaremos os dados da imagem desta extensão para a variável `image`, e os dados do cabeçalho para a variável `header`:

In [ ]:
image = header_data_unit_list[0].data
header = header_data_unit_list[0].header

Podemos imprimir o cabeçalho da imagem FITS na tela para que todo o seu conteúdo possa ser verificado ou utilizado. Observe que as informações do WCS para essas informações podem ser encontradas na parte inferior do cabeçalho impresso, abaixo.

In [ ]:
header

Observe que o cabeçalho *original* (conforme baixado do DSS) viola os padrões FITS WCS (porque inclui palavras-chave CDELTn e uma matriz de valores de CD; incluindo palavras-chave de matriz PC obsoletas). O cabeçalho foi limpo para estar em conformidade com os padrões existentes.

### Etapa 2: Leia no sistema de coordenadas da imagem FITS com astropy.wcs.WCS

Como o cabeçalho contém informações WCS e age como um dicionário Python, um objeto Astropy `WCS` pode ser criado diretamente do cabeçalho FITS.

In [ ]:
wcs_helix = WCS(header)

Vamos imprimir o objeto `WCS` para ver quais valores foram extraídos do cabeçalho.

In [ ]:
wcs_helix

### Etapa 3: Plote a nebulosa Helix com eixos de coordenadas do céu (RA e DEC)

Os dados da imagem, `image`, são uma matriz 2D de valores e, por si só, não contêm informações sobre as coordenadas do céu dos pixels. Assim, se plotarmos a imagem por si só, os eixos de plotagem mostrariam valores de pixel. (Estaremos usando a biblioteca `matplotlib` para a plotagem.)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(image, origin='lower', cmap='cividis')

Todas as informações que mapeiam esses valores de pixel para as coordenadas do céu vêm dos metadados WCS, que carregamos no objeto `wcs_helix` (do cabeçalho do arquivo FITS). Este objeto `WCS` é construído para que possa ser fornecido ao `matplotlib` com a palavra-chave `projection`, como mostrado na chamada para `matplotlib.pyplot.subplot` abaixo, a fim de produzir eixos que mostrem informações de coordenadas do céu. de valores de pixel. Também sobreporemos uma grade de coordenadas em coordenadas equatoriais ICRS passando o nome do quadro de coordenadas do céu (aqui, "icrs") para o método `ax.get_coords_overlay()`.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs_helix)
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'RA')
plt.ylabel(r'Dec')

overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')

## Exercício

Copie o bloco de código acima e, em vez disso, sobreponha uma grade de coordenadas em coordenadas galácticas.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs_helix)
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'RA')
plt.ylabel(r'Dec')

overlay = ax.get_coords_overlay('galactic')
overlay.grid(color='white', ls='dotted')

## Seção 3: Plote um marcador de escala em uma imagem com WCS

Para adicionar um marcador de escala (ou seja, uma linha de um tamanho *angular* específico) à imagem da nebulosa Helix, usaremos o método matplotlib `Axes.arrow` para desenhar uma linha.

Primeiro, precisamos decidir onde colocar a barra de escala. No exemplo abaixo, definimos o centro do marcador de escala em `(RA, Dec) = (337 graus, -21,2 graus)`.

Em seguida, usamos o atributo `transform` de `Axes.arrow` para desenhar nossas barras de escala em graus (em vez de coordenadas de pixel). Nesse caso, desenhamos um marcador de escala com um comprimento de 0,1 graus. As entradas do método de seta são `ax.arrow(x, y, dx, dy, **kwargs)`, com `x` e `y` sendo o `RA` e o `Dec` do início da linha. Usamos `dx=0` para que não haja componente horizontal na barra, e `dy=0.1`, que dá o comprimento da seta na direção vertical. Para garantir que a seta seja desenhada no quadro de coordenadas J2000 ICRS, passamos `ax.get_transform('icrs')` para a palavra-chave `transform`.

Finalmente, usamos `matplotlib.pyplot.text` para marcar o comprimento do marcador de escala.

In [ ]:
fig = plt.figure(figsize=(10, 10), frameon=False)
ax = plt.subplot(projection=wcs_helix)
ax.arrow(337, -21.2, 0, 0.1, 
         head_width=0, head_length=0, 
         fc='white', ec='white', width=0.003, 
         transform=ax.get_transform('icrs'))
plt.text(337.05, -21.18, '0.1 deg', 
         color='white', rotation=90, 
         transform=ax.get_transform('icrs'))
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'RA')
plt.ylabel(r'Dec')

## Exercício

Faça uma barra horizontal com o mesmo comprimento. Tenha em mente que o ângulo de 1 hora = 15 graus.